# ASR Notebook


### Load the dataset

In [ ]:
from datasets import load_dataset

# Load the Afrispeech dataset
dataset = load_dataset("tobiolatunji/afrispeech-200", "all", streaming=True)
print("Number of examples", len(dataset))

# Access the filtered examples
for example in dataset:
    print(example)

Number of examples 3
train
validation
test


### Dropping some of the unused columns

In [ ]:
# drop unused columns
dataset_cln = dataset.remove_columns(['speaker_id', 'path', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])

### Prepare Feature Extractor, Tokenizer and Data

- A feature extractor pre-processes the raw audio inputs
- The model performs the sequence-to-sequence mapping
- A tokenizer post-processes the model outputs to text format

load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

#### load WhisperTokenizer

The Whisper model outputs a sequence of token ids. The tokenizer maps each of these token ids to their corresponding text string. For the English language, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to specify the target language and the task. These arguments inform the tokenizer to prefix the language and task tokens to the start of encoded label sequences:

In [ ]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

#### Combine To create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can wrap both into a single WhisperProcessor class. This processor object inherits from WhisperFeatureExtractor and WhisperProcessor, and can be used on the audio inputs and model predictions as required. In doing so, we only need to keep track of two objects during training: the processor and the model:

In [ ]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

### Data Preparation

Since our input audio is sampled at 44100Hz, we need to downsample it to 16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's cast_column method. This operation does not change the audio in-place, but rather signals to datasets to resample audio samples on the fly the first time that they are loaded:

In [ ]:
from datasets import Audio
dataset_cln = dataset_cln.cast_column("audio", Audio(sampling_rate=16000))

## A function to prepare our data for the model:

We first provide set constants for the maximum duration of audio in seconds, the maximum input length, and the maximum label length. The maximum input length and label length are important for defining the constraints and processing requirements for our audio data and its corresponding transcriptions within the model.

We then load the resampled audio data by calling batch["audio"].
We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
# Pre-processing to suit the max audio length required by Whisper
MAX_DURATION_IN_SECONDS = 30.0
MAX_INPUT_LENGTH = MAX_DURATION_IN_SECONDS * 16000
MAX_LABEL_LENGTH = 448

# Add these functions to your code
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    batch["input_length"] = len(batch["audio"])
    batch["labels_length"] = len(tokenizer(batch["transcript"], add_special_tokens=False).input_ids)
    return batch
def filter_inputs(input_length):
    """Filter inputs with zero input length or longer than 30s"""
    return 0 < input_length < MAX_INPUT_LENGTH
def filter_labels(labels_length):
    """Filter label sequences longer than max length (448)"""
    return labels_length < MAX_LABEL_LENGTH

In [ ]:
dataset_cln = dataset_cln.map(prepare_dataset)
dataset_cln = dataset_cln.filter(filter_inputs, input_columns=["input_length"])
dataset_cln = dataset_cln.filter(filter_labels, input_columns=["labels_length"])
dataset_cln = dataset_cln.remove_columns(['labels_length', 'input_length'])

In [ ]:
val_dataset = dataset_cln['train'].take(3000)
train_dataset = dataset_cln['train'].skip(3000)

dataset_cln['train'] = train_dataset
dataset_cln['validation'] = val_dataset

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
#  initialising the data collator defined above:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate
metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
# Training Args
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./dsn_afrispeech",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    resume_from_checkpoint=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_cln['train'],
    eval_dataset=dataset_cln['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

Reading metadata...: 57819it [00:04, 14440.02it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,0.676100,0.683783,29.587860
1000,0.413400,0.663427,33.633950
1500,0.585800,0.615651,31.333195
2000,0.601700,0.584267,27.880180


Reading metadata...: 57819it [00:14, 3962.94it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Reading metadata...: 57819it [00:02, 25766.29it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warn

Step,Training Loss,Validation Loss,Wer
500,0.676100,0.683783,29.587860
1000,0.413400,0.663427,33.633950
1500,0.585800,0.615651,31.333195
2000,0.601700,0.584267,27.880180
2500,0.579800,0.541273,26.759927
3000,0.518000,0.526127,26.411613


Reading metadata...: 57819it [00:01, 30339.38it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Reading metadata...: 57819it [00:01, 29461.27it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  war